In [ ]:
# !pip install tensorflow --break-system-packages
# !pip install nltk --break-system-packages
# !pip install numpy --break-system-packages
# !pip install pandas --break-system-packages
# !pip install gensim --break-system-packages
# !pip install tqdm --break-system-packages
# !pip install scipy --break-system-packages

In [ ]:
from gensim.models import Word2Vec
import nltk
import random
from tqdm import tqdm
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import brown
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
from scipy.stats import spearmanr
from math import sqrt
nltk.download('punkt')
nltk.download('brown')

In [ ]:
def get_cosine(vector1, vector2):
    intersection = min(len(vector1), len(vector2))
    numerator = sum(vector1[i] * vector2[i] for i in range(intersection))
    sum1 = sum(vector1[i] ** 2 for i in range(len(vector1)))
    sum2 = sum(vector2[i] ** 2 for i in range(len(vector2)))
    denominator = sqrt(sum1) * sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
def reduce_dimensionality(original_vector, new_length):
    # Assuming original_vector is a 1D NumPy array
    original_vector = original_vector.reshape(1, -1)  # Convert to a 2D array
    pca = PCA(n_components=new_length)
    reduced_vector = pca.fit_transform(original_vector)
    reduced_vector = reduced_vector.flatten()  # Convert back to a 1D array

    return reduced_vector

In [ ]:
sentences = brown.sents()

tokenized_sentences = [word_tokenize(' '.join(sentence)) for sentence in sentences]

model = Word2Vec(sentences=tokenized_sentences, vector_size=300, window=5, min_count=1, workers=4)
import pandas as pd
simlex_subset = pd.read_csv('./SimLex-999.txt', sep='\t')
word1_list = simlex_subset['word1'].tolist()
word2_list = simlex_subset['word2'].tolist()
predictions=[]
for i in tqdm(range(0,len(word1_list))):
    vector1 = model.wv[word1_list[i]] if word1_list[i] in model.wv else np.zeros(model.vector_size)
    vector2 = model.wv[word2_list[i]] if word2_list[i] in model.wv else np.zeros(model.vector_size)
    if np.any(np.isnan(vector1)) or np.any(np.isnan(vector2)) or np.all(vector1 == 0) or np.all(vector2 == 0):
        similarity = 0
    else:
        similarity = get_cosine(vector1,vector2)

    predictions.append(similarity)
ground_truth_scores = simlex_subset['SimLex999'].values
correlation, _ = spearmanr(predictions, ground_truth_scores)
print(f'Spearman Correlation: {correlation*100}%')